In [5]:
# 기본
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# 그래프 설정
plt.rcParams['font.family'] = 'Malgun Gothic'

# 경고 뜨지 않게 설정
import warnings
warnings.filterwarnings('ignore')

### 파일패스 설정

In [6]:
# 이부분 확인하기
data_type = "train"
drive_folder = f'../data/open/{data_type}/3.승인매출정보/'
month = "07"
category = "승인매출정보"

In [7]:
segment_folder = f'../data/open/{data_type}/1.회원정보/'
segment_category = "회원정보"

dict1 = {
    '09.미사용' : 0,
    '05.10만원-' : 1,
    '04.10만원+' : 2,
    '03.30만원+' : 3,
    '02.50만원+' : 4,
    '01.100만원+' : 5
}

### 데이터 불러오기

In [8]:
file_name=f"{drive_folder}2018{month}_{data_type}_{category}.parquet"
df=pd.read_parquet(file_name)

### 기본 전처리

In [9]:
# 모든 값이 동일한 컬럼 제거 (기준년도 컬럼 제외)
constant_cols = [col for col in df.columns.difference(['기준년월']) if df[col].nunique() == 1]
if constant_cols:
    print(f"🧹 동일값 컬럼 제거: {constant_cols}")
    df = df.drop(columns=constant_cols)

# 결측치가 있는 컬럼 제거
null_cols = df.columns[df.isnull().any()]
if len(null_cols) > 0:
    print(f"🧹 결측치 컬럼 제거: {list(null_cols)}")
    df = df.drop(columns=null_cols)

# 세그먼트 컬럼 추가
if data_type=="train":
    segment_df = pd.read_parquet(f"{segment_folder}2018{month}_{data_type}_{segment_category}.parquet")
    print(f"🔀 세그먼트 병합")
    df = df.merge(segment_df[['ID', 'Segment']], on='ID', how='left')
    df=pd.get_dummies(df, columns=['Segment'])

# 범주형 인코딩
df['이용금액대'] = df['이용금액대'].map(dict1)

df

🧹 동일값 컬럼 제거: ['RP건수_건강_B0M', 'RP건수_유선방송_B0M', 'RP후경과월_건강', 'RP후경과월_유선방송', '납부_건강연금이용금액', '납부_렌탈료이용금액', '납부_유선방송이용금액', '승인거절건수_기타_B0M', '승인거절건수_입력오류_B0M', '여유_여행이용금액', '이용개월수_당사페이_R6M', '이용건수_당사기타_B0M', '이용건수_당사기타_R3M', '이용건수_당사기타_R6M', '이용건수_당사페이_B0M', '이용건수_당사페이_R3M', '이용건수_당사페이_R6M', '이용건수_부분무이자_B0M', '이용금액_당사기타_B0M', '이용금액_당사기타_R3M', '이용금액_당사기타_R6M', '이용금액_당사페이_B0M', '이용금액_당사페이_R3M', '이용금액_당사페이_R6M', '이용금액_부분무이자_B0M', '증감_RP건수_건강_전월', '증감_RP건수_렌탈_전월', '증감_RP건수_보험_전월', '증감_RP건수_아파트_전월', '증감_RP건수_유선방송_전월', '증감_RP건수_학습비_전월', '할부건수_부분_14M_R12M', '할부건수_부분_3M_R12M', '할부건수_부분_6M_R12M', '할부금액_부분_3M_R12M', '할부금액_부분_6M_R12M']
🧹 결측치 컬럼 제거: ['_1순위업종', '_2순위업종', '_3순위업종', '_1순위쇼핑업종', '_2순위쇼핑업종', '_3순위쇼핑업종', '_1순위교통업종', '_2순위교통업종', '_3순위교통업종', '_1순위여유업종', '_2순위여유업종', '_3순위여유업종', '_1순위납부업종', '_2순위납부업종', '_3순위납부업종', '최종카드론_금융상환방식코드', '최종카드론_신청경로코드', '최종카드론_대출일자']
🔀 세그먼트 병합


,기준년월,ID,최종이용일자_기본,최종이용일자_신판,최종이용일자_CA,최종이용일자_카드론,최종이용일자_체크,최종이용일자_일시불,최종이용일자_할부,이용건수_신용_B0M,...,승인거절건수_한도초과_R3M,승인거절건수_BL_R3M,승인거절건수_입력오류_R3M,승인거절건수_기타_R3M,이용금액대,Segment_A,Segment_B,Segment_C,Segment_D,Segment_E
0,201807,TRAIN_000000,20180719,20180713,20180719,10101,20180203,20180709,20180713,11,...,3,0,0,0,5,False,False,False,True,False
1,201807,TRAIN_000001,20180719,20180719,20170728,20170327,10101,20180719,20171231,13,...,3,0,0,0,3,False,False,False,False,True
2,201807,TRAIN_000002,20180706,20180706,20180706,20151119,20141230,20180706,20180627,12,...,0,0,0,0,5,False,False,True,False,False
3,201807,TRAIN_000003,20180721,20180715,20180721,10101,20141111,20180704,20180715,6,...,3,0,0,0,5,False,False,False,True,False
4,201807,TRAIN_000004,20180124,20180124,10101,10101,20180512,20180124,10101,-2,...,0,0,0,0,0,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399995,201807,TRAIN_399995,20170217,20170217,10101,10101,20180719,20170217,20160501,0,...,0,0,0,0,0,False,False,False,False,True
399996,201807,TRAIN_399996,20180731,20180731,10101,20170112,10101,20180731,20180112,21,...,0,0,0,0,5,False,False,False,True,False
399997,201807,TRAIN_399997,20180726,20180726,10101,10101,20131124,20180720,20180726,16,...,0,0,0,0,4,False,False,True,False,False
399998,201807,TRAIN_399998,20161224,20161224,10101,10101,10101,20161224,20150122,-1,...,0,0,0,0,0,False,False,False,False,True


In [10]:
output_file=f"{drive_folder}2018{month}_processed__{category}.csv"
df.to_csv(output_file, index=False)